# Resolve did:btc1 from local Bitcoin node in regtest

In [3]:
did_to_resolve = "did:btc1:regtest:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65"

## Parse did:btc1 identifier

In [10]:
import re
match = re.search(r"^did:btc1:(?:(\d+):)?(?:(mainnet|signet|testnet|regtest):)?((k1|x1)[023456789acdefghjklmnpqrstuvwxyz]*)$", did_to_resolve)


In [15]:
match.groups()[3]

'k1'

## Resolve Initial Document

## Resolve Target Document

In [62]:
import asyncio

from bitcoinrpc import BitcoinRPC

In [63]:
rpc = BitcoinRPC.from_config("http://localhost:18443", ("polaruser", "polarpass"))
await rpc.getconnectioncount()

0